In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf

In [2]:
import sys
from pathlib import Path
print(Path.cwd())


/Users/vaishnukanna/Documents/jegadeesh-titman-momentum/results


In [3]:
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [4]:
from src.capm_reg import capm_regression
from src.config import DATA_DIR

In [5]:
path1 = DATA_DIR / "portfolio" / "J6_K6_skip0_returns.csv"
if not path1.exists():
        raise FileNotFoundError(f"Portfolio data not found at {path1}. Run portfolio.py first.")
else:
    portfolio = pd.read_csv(
        path1, 
        parse_dates=["Date"]
        ).set_index("Date")
    print(portfolio.head())

              winner     loser    spread
Date                                    
2001-02-01  0.019540 -0.192802  0.212342
2001-03-01 -0.043318 -0.035796 -0.007522
2001-04-01  0.077074  0.216275 -0.139201
2001-05-01  0.038592 -0.010052  0.048643
2001-06-01  0.033504  0.051841 -0.018337


In [6]:
path2 = DATA_DIR / "portfolio" / "market_returns.csv"
if not path2.exists():
    raise FileNotFoundError(f"Risk-free rate data not found at {path2}. Run capm_params.py first.")
else:
    market = pd.read_csv(
        path2,
        parse_dates=["Date"]
        ).set_index("Date")["Market_Returns"]

    print(market.head())

Date
2000-02-01   -0.020108
2000-03-01    0.096720
2000-04-01   -0.030796
2000-05-01   -0.021915
2000-06-01    0.023934
Name: Market_Returns, dtype: float64


In [7]:
path3 = DATA_DIR / "portfolio" /"rf_rate.csv"
if not path3.exists():
    raise FileNotFoundError(f"Risk-free rate data not found at {path3}. Run capm_params.py first.")
else:
    rf = pd.read_csv(
        path3,
        parse_dates=["Date"]
    ).set_index("Date")["Rf_Rate"]

    print(rf.head())

Date
2001-07-01    0.003008
2001-08-01    0.002900
2001-09-01    0.002192
2001-10-01    0.001867
2001-11-01    0.001633
Name: Rf_Rate, dtype: float64


In [8]:
data = pd.concat(
    [portfolio, market, rf],
    axis=1
).dropna()

data.head()

,winner,loser,spread,Market_Returns,Rf_Rate
Date,,,,,
2001-07-01,0.003377,-0.027695,0.031072,-0.010740,0.003008
2001-08-01,-0.025843,-0.096090,0.070247,-0.064108,0.002900
2001-09-01,-0.142355,-0.196531,0.054176,-0.081723,0.002192
2001-10-01,0.145238,0.113987,0.031251,0.018099,0.001867
2001-11-01,0.105253,0.205434,-0.100181,0.075176,0.001633


In [9]:
winner = data["winner"]
loser = data["loser"]
spread = data["spread"]
market = data["Market_Returns"]
rf = data["Rf_Rate"]

In [10]:
print(type(winner))
print(type(market))
print(type(rf))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [11]:
capm_winner = capm_regression(
    portfolio_returns=winner,
    market_returns=market,
    risk_free_rate=rf
)

capm_winner

const    0.019388
Rm       1.072583
dtype: float64
Index(['const', 'Rm'], dtype='object')


{'alpha': np.float64(0.01938816487694194),
 'beta': np.float64(1.0725833827393805),
 't_alpha': np.float64(9.90508883515461),
 't_beta': np.float64(24.247895458397025),
 'r_squared': np.float64(0.6869014158784339)}

In [12]:
capm_loser = capm_regression(
    portfolio_returns=loser,
    market_returns=market,
    risk_free_rate=rf
)

capm_loser

const    0.002044
Rm       1.563828
dtype: float64
Index(['const', 'Rm'], dtype='object')


{'alpha': np.float64(0.0020442814762786287),
 'beta': np.float64(1.5638279181552777),
 't_alpha': np.float64(0.855295518039059),
 't_beta': np.float64(28.95248071305359),
 'r_squared': np.float64(0.7577392675596328)}

In [13]:
capm_strategy = capm_regression(
    portfolio_returns=spread,
    market_returns=market,
    risk_free_rate=rf
)

capm_strategy

const    0.016201
Rm      -0.489586
dtype: float64
Index(['const', 'Rm'], dtype='object')


{'alpha': np.float64(0.016200556760477512),
 'beta': np.float64(-0.4895864364051691),
 't_alpha': np.float64(5.2366811929823065),
 't_beta': np.float64(-7.002883291511312),
 'r_squared': np.float64(0.15468179561730522)}

In [ ]:
capm_table = pd.DataFrame({
    "Winner": capm_winner,
    "Loser": capm_loser,
    "Winner-Loser": capm_strategy
}).T

In [15]:
capm_table

,alpha,beta,t_alpha,t_beta,r_squared
Winner,0.019388,1.072583,9.905089,24.247895,0.686901
Loser,0.002044,1.563828,0.855296,28.952481,0.757739
Winner-Loser,0.016201,-0.489586,5.236681,-7.002883,0.154682


In [16]:
capm_table.to_csv(PROJECT_ROOT/"results/capm.csv")